In [ ]:
import polars as pl

multiple_choice_cols = [
    "Languages",
    "Dependency management scheme",
    "Additional dependency features",
]
categorical_cols = ["Code available"]
df_annots = pl.read_csv("data/papers_annotated.csv").with_columns(
    # Column transformations for multiple-choice variables
    pl.col(multiple_choice_cols).str.split(", ").cast(pl.List(pl.Categorical)),
    pl.col(categorical_cols).cast(pl.Categorical),
    pl.col("Dependency versioning").replace_strict(
        {
            "Versioned": True,
            "Unversioned": False,
        }
    ),
)
df_annots.head()

In [ ]:
def summary_counts(df: pl.DataFrame, col: str) -> pl.DataFrame:
    """Generate summary counts for a given column."""
    return df.select(pl.col(col).value_counts(sort=True, name="Count").struct.unnest()).with_columns(
        proportion=pl.col("Count") / pl.col("Count").sum()
    )

In [ ]:
# Generate summary of "Code available" counts
df_annots.pipe(summary_counts, "Code available")

In [ ]:
# Domains of repository and git URLs -- Where are artifacts hosted?
df_domains = (
    df_annots.with_columns(
        # Append columns "Repository URL" and "Git URL" with their domains
        git_domain=pl.col("Git URL").str.extract(r"https?://([^/]+)"),
        repo_domain=pl.col("Repository URL").str.extract(r"https?://([^/]+)"),
    )
    .with_columns(
        # Join with plus sign (e.g. "github.com + zenodo.org")
        domain_combo=pl.concat_str(
            [pl.col("git_domain"), pl.col("repo_domain")],
            separator=" + ",
            ignore_nulls=True,
        ),
    )
    .filter(pl.col("repo_domain").is_not_null() | pl.col("git_domain").is_not_null())
)

# Generate summary of domain combinations
df_domains.pipe(summary_counts, "domain_combo")

In [ ]:
# Summary counts of Languages and Dependency mgmt
df_code_available = df_annots.filter(pl.col("Code available") == "Yes")
n = df_code_available.height
df_code_available.explode("Languages").pipe(summary_counts, "Languages").with_columns(
    proportion_of_projects=pl.col("Count") / n
)

In [ ]:
df_code_available["Dependency management scheme"].explode().unique()

In [ ]:
# Dependency management schemes in order of least to most precise
# Hashfile is more precise than requirements.txt etc.
deps_schemes = [
    "None (import only)",
    "In README",
    "In comment",
    "In notebook cell",
    "Requirements file",
    "Lockfile",
    "Dependency-free",
]
df_deps = df_code_available.with_columns(
    # Add column: Most precise dependency management scheme.
    pl.col("Dependency management scheme")
    .cast(pl.List(pl.Enum(deps_schemes)))
    .list.sort(descending=True)
    .list.first()
    .alias("Most precise dependency management scheme"),
)
with pl.Config(tbl_rows=50):
    print(
        df_deps.select(
            "Name",
            "Dependency management scheme",
            "Additional dependency features",
            "Most precise dependency management scheme",
            "Dependency versioning",
        )
    )

In [ ]:
pl.Config.set_tbl_rows(20)
n_unversioned = df_deps.filter(pl.col("Dependency versioning").not_()).height
var = "Most precise dependency management scheme"
df_deps.pipe(summary_counts, var).with_columns(
    proportion_of_projects=pl.col("Count") / n,
    proportion_of_unversioned=pl.col("Count") / n_unversioned,
)

In [ ]:
# Machine-actionability: Can the machine recreate the environment?
# This excludes manual steps like "dependencies in readme"

# Pick from these
machine_actionable_features = [
    "Requirements file",
    "Lockfile",
    "Dependency-free",
]
df_deps = df_deps.with_columns(
    # Mark machine-actionability (boolean)
    machine_actionable=pl.col("Dependency management scheme")
    .list.eval(pl.element().is_in(machine_actionable_features))
    .list.any()
    & pl.col("Dependency versioning")
).rename({"machine_actionable": "Machine-actionable"})

In [ ]:
# How many were machine-actionable?
df_deps.pipe(summary_counts, "Machine-actionable")
# df_deps.filter(pl.col("Machine-actionable"))

In [ ]:
# Specifically, only N projects’ dependencies could be considered reproducible as well as machine-actionable, and only n of these used requirements.txt as their sole dependency pinning scheme.
df_deps.filter(pl.col("Machine-actionable")).pipe(summary_counts, "Most precise dependency management scheme")

In [ ]:
# requirements.txt: You’re holding it wrong
# How many projects using requirements.txt (as their best deps mgmt scheme) were actually properly versioned?
df_deps.filter(
    pl.col("Most precise dependency management scheme") == "Requirements file"
).pipe(summary_counts, "Machine-actionable")

## LaTeX tables 

In [ ]:
import os

# Prepare output directory
tables_path = "outputs/tables/"
os.makedirs(tables_path, exist_ok=True)

In [ ]:
# Source code availability
order = [
    "Yes",
    "No",
    "Data only",
    "Unfulfilled: No reference",
    "Unfulfilled: Empty",
    "Not applicable",
]

tab_available = (
    df_annots.pipe(summary_counts, "Code available")
    .with_columns(
        pl.col("Code available").cast(pl.Enum(order)),
        (pl.col("proportion") * 100).alias("\\% of projects"),
    )
    .drop("proportion")
    .sort("Code available")
    .with_columns(
        pl.col("Code available")
        .cast(pl.String)
        .replace(
            {
                "Unfulfilled: No reference": "\\quad of which provided no location",
                "Unfulfilled: Empty": "\\quad of which were empty",
                "Data only": "\\quad of which provided data only",
            }
        )
    )
    .rename({"Code available": "Source code availability"})
)
print(tab_available)

tab_available.to_pandas().to_latex(
    f"{tables_path}tab_available.tex",
    caption=f"Source code availability among the {df_annots.height} annotated projects",
    label="tab:available",
    position="htbp",
    float_format="%.0f",
    index=False,
    bold_rows=True,
    escape=False,
)

In [ ]:
# Table: Programming languages used in the projects
tab_langs = (
    df_code_available.explode("Languages")
    .pipe(summary_counts, "Languages")
    .select(
        pl.col("Languages").alias("Programming language"),
        pl.col("Count"),
        # (pl.col("proportion") * 100).alias("% of languages"),
        ((pl.col("Count") / df_code_available.height) * 100).alias("% of projects")
    )
)

tab_langs.to_pandas().to_latex(
    f"{tables_path}tab_langs.tex",
    caption=f"Programming languages used in the {df_code_available.height} projects with available code",
    label="tab:langs",
    column_format="lrrr",
    position="htbp",
    float_format="%.0f",
    index=False,
    bold_rows=True,
    escape=True
)

In [ ]:
tab_schemes = (
    df_deps.pipe(summary_counts, "Most precise dependency management scheme")
    .with_columns(
        (pl.col("proportion") * 100).alias("% of projects"),
    )
    .rename(
        {
            "Most precise dependency management scheme": "Dependency management scheme",
        }
    )
    .drop("proportion")
)

tab_schemes.to_pandas().to_latex(
    f"{tables_path}tab_schemes.tex",
    caption=f"Highest-fidelity dependency management schemes used in the {df_code_available.height} projects with available code",
    label="tab:schemes",
    column_format="lrrr",
    position="htbp",
    float_format="%.0f",
    index=False,
    bold_rows=True,
    escape=True,
)